<a href="https://colab.research.google.com/github/Khushil-Modi/cm1_python_test/blob/main/Khushil_Modi_July_12th_2023_CM1_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Test Instructions
The purpose of this simple coding test is to allow us to get a grasp on the quality of your code and to identify the role that would suit you best within our team. You don't NEED to be able to answer all of the items, but the ones that you do answer need to be correct. Feel free to use Google or any other tools that you prefer to complete these tasks.

Install any needed third-party libraries below this block. Please install the minimum amount of libraries you need.

!pip install whatever_you_need

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.0 MB/s eta 0:00:00


### Task 1 (Data manipulation / Visualization)
Download the file: https://drive.google.com/file/d/1R_M3xI3b_BqAN_xWLqJDneg273X_sok5/view?usp=sharing

In `message_data.csv` there's two columns, one corresponds to the unique id of someone sending a message and the other one to when they sent it.

Generate an interactive chart that shows the percentage of daily messages sent by new users (those who sent their first message ever on a given day).

In [3]:
import pandas as pd
import plotly.express as px

# Load the message data from CSV
df = pd.read_csv('/content/message_data.csv')

# Convert the 'sent_at' column to datetime
df['message_time'] = pd.to_datetime(df['message_time'])

# Sort the dataframe by 'sent_at' in ascending order
df = df.sort_values('message_time')

# Group the data by date and count the number of unique users per day
daily_users = df.groupby(df['message_time'].dt.date)['author_id'].nunique()

# Calculate the daily percentage of new users
daily_percentage = daily_users / df['author_id'].nunique() * 100

# Create a new dataframe with the daily date and corresponding percentage
df_percentage = pd.DataFrame({'Date': daily_percentage.index, 'Percentage': daily_percentage.values})

# Create an interactive line chart using Plotly
fig = px.line(df_percentage, x='Date', y='Percentage', title='Percentage of Daily Messages Sent by New Users')
fig.show()


### Task 2 (Django)
**To take the django portion of this test go to: https://github.com/llu13701/cm1_python_test and follow the instructions.**

### Task 3 (LangChain)
Write a simple Chain that does the following:
- Uses openai's gpt-3.5-turbo model.
- Remembers the last 2 message exchanges, as well as the system message.
- Greets people and tells a joke about their name.

In [9]:
import openai

# Set up OpenAI API credentials
openai.api_key = 'YOUR_API_KEY'

# Define a function to generate a response using the chat model
def generate_response(input_text):
    model_prompt = "GPT-3.5-turbo is great at conversations!\n\nUser: " + input_text + "\nAI:"
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=model_prompt,
        max_tokens=50,
        temperature=0.7,
        n = 1,
        stop=None
    )
    return response.choices[0].text.strip()

# Define a function to handle the conversation
def handle_conversation(message):
    user_message = message['content']
    system_message = None

    # Check if a system message is present
    if 'system' in message:
        system_message = message['system']

    # Remember the last two exchanges
    if 'messages' not in message:
        message['messages'] = [{'role': 'system', 'content': system_message}]
    message['messages'].append({'role': 'user', 'content': user_message})
    message['messages'] = message['messages'][-2:]

    # Generate a response
    response = generate_response(user_message)

    # Extract the user's name
    user_name = None
    for msg in message['messages']:
        if msg['role'] == 'user':
            if 'name' in msg:
                user_name = msg['name']
            break

    # Greet and tell a joke about the user's name
    if user_name:
        response += f" Hello, {user_name}! Here's a joke for you: Why don't scientists trust atoms? Because they make up everything!"

    return {'message': {'role': 'assistant', 'content': response}}

# Example usage
conversation = {
    'messages': [
        {'role': 'system', 'content': 'You are a bot'},
        {'role': 'user', 'content': 'Hello, my name is John'}
    ]
}

response = handle_conversation(conversation['messages'][-1])
print(response['message']['content'])


RateLimitError: ignored

### Task 4 (Discord API)

Write a very simple discord bot that says "Hello {user_name}!" to every new incoming message in a discord server.


In [11]:
import discord

intents = discord.Intents.default()
intents.messages = True

# Create a new Discord client
client = discord.Client(intents=intents)

# Event handler for when the bot is ready
@client.event
async def on_ready():
    print(f'Bot is ready! Logged in as {client.user}')

# Event handler for when a new message is received
@client.event
async def on_message(message):
    if message.author == client.user:
        return

    # Send a response with a greeting
    response = f"Hello {message.author.name}!"
    await message.channel.send(response)

# Run the bot
client.run('YOUR_DISCORD_BOT_TOKEN')


RuntimeError: ignored

### Task 5
Use LangChain to write a discord bot that will:
- Greet a user, making a joke about their name and then proceed to answer any question they have, whenever a user sends a message.
- Use async with langchain chain calls.
